# DeepSphere using ModelNet40 dataset
### Benchmark with Cohen method S2CNN[[1]](http://arxiv.org/abs/1801.10130) and Esteves method[[2]](http://arxiv.org/abs/1711.06721) and others spherical CNNs
Multi-class classification of 3D objects, using the interesting property of rotation equivariance.

The 3D objects are projected on a unit sphere.
Cohen and Esteves use equiangular sampling, while our method use a HEAlpix sampling

Several features are collected:
* projection ray length (from sphere border to intersection [0, 2])
* cos/sin with surface normal
* same features using the convex hull of the 3D object

### This notebook put forward the problem concerning the rotation perturbations

## 0.1 Load libs

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import shutil
import sys
sys.path.append('../../')

os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # change to chosen GPU to use, nothing if work on CPU

import numpy as np
import time
import matplotlib.pyplot as plt
import healpy as hp

In [ ]:
from deepsphere import models, experiment_helper, plot, utils
from deepsphere.data import LabeledDatasetWithNoise, LabeledDataset
import hyperparameters

from load_MN40 import plot_healpix_projection, ModelNet40DatasetTF, ModelNet40DatasetCache

## 0.2 Define parameters

In [ ]:
Nside = 32
datapath = '../../../data/ModelNet40/' # localisation of the .OFF files
exp = 'rot'
proc_path = datapath[1:]

In [ ]:
augmentation = 3        # number of element per file (1 = no augmentation of dataset)
nfeat = 6

In [ ]:
plot_healpix_projection(datapath+'airplane'+"/train/"+'airplane'+"_0069.off", 32, rotp = True, rot = (90,0,0))

## 1 Load dataset

Train datasets

In [ ]:
train_rot_dataset = ModelNet40DatasetCache(datapath, 'train', nside=Nside, nfeat=nfeat, augmentation=3, nfile=None, 
                                           experiment='deepsphere_rot_notr')


In [ ]:
train_rot_tr_dataset = ModelNet40DatasetCache(datapath, 'train', nside=Nside, nfeat=nfeat, augmentation=3, nfile=None, 
                                              experiment='deepsphere_rot')


In [ ]:
train_tr_dataset = ModelNet40DatasetCache(datapath, 'train', nside=Nside, nfeat=nfeat, augmentation=3, nfile=None, 
                                          experiment='deepsphere')


In [ ]:
train_dataset = ModelNet40DatasetCache(datapath, 'train', nside=Nside, nfeat=nfeat, augmentation=1, nfile=None, 
                                       experiment='deepsphere_notr')


Test datasets

In [ ]:
test_tr_dataset = ModelNet40DatasetCache(datapath, 'test', nside=Nside, nfeat=nfeat, augmentation=3, nfile=None)

In [ ]:
test_dataset = ModelNet40DatasetCache(datapath, 'test', nside=Nside, nfeat=nfeat, augmentation=1, nfile=None,
                                        experiment='deepsphere_notr')

In [ ]:
test_rot_tr_dataset = ModelNet40DatasetCache(datapath, 'test', nside=Nside, 
                                       nfeat=nfeat, experiment='deepsphere_rot', augmentation=3, nfile=None)

In [ ]:
test_rot_dataset = ModelNet40DatasetCache(datapath, 'test', nside=Nside, 
                                       nfeat=nfeat, experiment='deepsphere_rot_notr', augmentation=3, nfile=None)

Tensorflow pipeline datasets

In [ ]:
train_TFDataset = ModelNet40DatasetTF(datapath, 'train', nside=Nside,
                                      nfeat=nfeat, augmentation=augmentation, nfile=None, 
                                      experiment='deepsphere'+('_rot' if exp=='rot' else '')+('_notr' if 'pert' not in exp else ''))

Normalize data

In [ ]:
# from ModelNet40.load_MN40 import compute_mean_std

In [ ]:
# compute_mean_std(test_rot_dataset, 'test', datapath, Nside)

In [ ]:
def transform(data, phi=None, theta=None):
    batch_size, npix, nfeat = data.shape
    if theta is None or phi is None:
        phi = np.random.rand() * 2 * np.pi
        theta = np.random.rand() * np.pi
    nside = hp.npix2nside(npix)

    # Get theta, phi for non-rotated map
    t,p = hp.pix2ang(nside, np.arange(npix), nest=True) #theta, phi

    # Define a rotator
    r = hp.Rotator(deg=False, rot=[phi, theta])

    # Get theta, phi under rotated co-ordinates
    trot, prot = r(t,p)

    # Interpolate map onto these co-ordinates
    new_data = np.zeros(data.shape)
    for b in range(batch_size):
        for f in range(nfeat):
            new_data[b,:,f] = hp.get_interp_val(data[b,:,f], trot, prot, nest=True)

    return new_data

In [ ]:
def transform_equator(data):
    return transform(data, 0, np.pi/2).astype(np.float32)

In [ ]:
def transform_shift(data):
    batch_size, npix, nfeat = data.shape
    new_data = data.copy()
    nside = hp.npix2nside(npix)
    theta, _ = hp.pix2ang(nside, range(npix))
    theta_u = np.unique(theta)
    for b in range(batch_size):
        for f in range(nfeat):
            new_data[b, :, f] = hp.reorder(data[b, :, f], n2r=True)
            for t in theta_u:
                ligne_ind = np.where(theta==t)[0]
                ligne_ind_roll = np.roll(ligne_ind, len(ligne_ind)//4)
                new_data[b,ligne_ind_roll,f] = new_data[b,ligne_ind,f]
            new_data[b, :, f] = hp.reorder(new_data[b, :, f], r2n = True)
    return new_data

In [ ]:
def transform_inverse(data):
    """
    180° rotation around X-axis
    """
    batch_size, npix, nfeat = data.shape
    data_c = data.copy()
    new_data = data.copy()
    new_data[:] = -10
    nside = hp.npix2nside(npix)
    theta, _ = hp.pix2ang(nside, range(npix))
    theta_u = np.unique(theta)
    for b in range(batch_size):
        for f in range(nfeat):
            data_c[b, :, f] = hp.reorder(data[b, :, f], n2r=True)
            for i, (t, t_end) in enumerate(zip(theta_u, theta_u[::-1])):
                ligne_ind = np.where(theta==t)[0]
                ligne_ind_roll = np.where(theta==t_end)[0][::-1]
                if i > len(theta_u)/4 and i < len(theta_u)*3/4:
                    ligne_ind_roll = np.roll(ligne_ind_roll, (i+1)%2)
                new_data[b,ligne_ind_roll,f] = data_c[b,ligne_ind,f]
            new_data[b, :, f] = hp.reorder(new_data[b, :, f], r2n = True)
    return new_data

In [ ]:
hp.orthview(im1[:,0], rot=(0,0,0), title=test_rot_dataset.classes[label[0]], nest=True, cmap=cm, min=cmin, max=cmax)
plt.figure()
im2 = transform_shift(im1[np.newaxis,:,:])
hp.orthview(im2[0,:,0], rot=(0,0,0), title=test_rot_dataset.classes[label[0]], nest=True, cmap=cm, min=cmin, max=cmax)

In [ ]:
from tqdm import tqdm
steps = train_S2_dataset.N // 1 + 1
data_iter = train_S2_dataset.iter(1)
cm = plt.cm.RdBu_r
cm.set_under('w')
for i in tqdm(range(steps)):
    data, label = next(data_iter)
    im1 = data[0,:,0]
    if np.std(im1)>4:
        print(np.std(im1))
        cmin = np.nanmin(im1)
        cmax = np.nanmax(im1)
        hp.orthview(im1, rot=(0,0,0), title=train_dataset.classes[label[0]], nest=True, cmap=cm, min=cmin, max=cmax)
        plt.figure()
#     if i > 1000:
#         break

In [ ]:
nclass = train_TFDataset.nclass
num_elem = train_TFDataset.N
print('number of class:',nclass,'\nnumber of elements:',num_elem)

## 2 Classification using DeepSphere

In [ ]:
EXP_NAME = 'MN40_{}_{}feat_{}aug_{}sides'.format(exp, nfeat, augmentation, Nside)

Load model with hyperparameters chosen.
For each experiment, a new EXP_NAME is chosen, and new hyperparameters are stored.
All informations are present 'DeepSphere/ModelNet40/resultsmn40.md'

In [ ]:
params = hyperparameters.get_params_shrec17_optim(train_TFDataset.N, EXP_NAME, Nside, nclass, 
                                                  nfeat_in=nfeat, architecture=experiment_type)
params["tf_dataset"] = train_TFDataset.get_tf_dataset(params["batch_size"])
# params["std"] = [0.001, 0.005, 0.0125, 0.05, 0.15, 0.5]
# params["full"] = [True]*6
model = models.deepsphere(**params)

In [ ]:
shutil.rmtree('summaries/{}/'.format(EXP_NAME), ignore_errors=True)
shutil.rmtree('checkpoints/{}/'.format(EXP_NAME), ignore_errors=True)

## 2.2 Train Network

In [ ]:
print("the number of parameters in the model is: {:,}".format(model.get_nbr_var()))

In [ ]:
accuracy_validation, loss_validation, loss_training, t_step, t_batch = model.fit(train_TFDataset, 
                                                                                 test_dataset, 
                                                                                 use_tf_dataset=True, cache=True)

In [ ]:
plot.plot_loss(loss_training, loss_validation, t_step, params['eval_frequency'])

Remarks

In [ ]:
model.evaluate(train_rot_dataset, None, cache=True)

In [ ]:
model.evaluate(train_rot_tr_dataset, None, cache=True)

In [ ]:
model.evaluate(train_dataset, None, cache=True)

In [ ]:
model.evaluate(train_tr_dataset, None, cache=True)

## test network

In [ ]:
model.evaluate(test_dataset, None, cache=True)

In [ ]:
test_dataset.set_transform(transform_shift)
print(model.evaluate(test_dataset, None, cache=True))
test_dataset.set_transform(None)

In [ ]:
model.evaluate(test_rot_dataset, None, cache=True)

In [ ]:
model.evaluate(test_rot_tr_dataset, None, cache=True)

In [ ]:
model.evaluate(test_tr_dataset, None, cache=True)

In [ ]:
test_no_dataset.set_transform(transform_equator)
print(model.evaluate(test_no_dataset, None, cache=True))
test_no_dataset.set_transform(None)

## Find differences

In [ ]:
labels_test = test_rot_dataset.get_labels()

In [ ]:
predictions, loss = model.predict(test_rot_dataset, None, cache=True)
print(loss)

In [ ]:
elem_ = np.arange(len(labels_test))%3

In [ ]:
files = test_rot_no_dataset.files[labels_test != predictions]
class_pred = np.asarray(test_rot_no_dataset.classes)[predictions[labels_test != predictions].astype(int)]

In [ ]:
for i, file in enumerate(files[-10:]):
    suffix = os.path.splitext(os.path.split(file)[-1])[0]
    data = test_rot_dataset.cache_npy(file, pick_randomly=False, 
                                         repeat=test_rot_no_dataset.repeat, experiment=test_rot_no_dataset.experiment)
    data2 = test_dataset.cache_npy(file, pick_randomly=False, 
                                         repeat=test_dataset.repeat, experiment=test_dataset.experiment)
    im1 = data[elem_[i]]
    im2 = data2[0]
#     print(np.std(im1))
#     print(np.std(im2))
#     print(class_pred[i])
    cmin = 0
    cmax = 1
    hp.orthview(im1[:,0], rot=(0,0,0), title=suffix, nest=True, cmap=cm, min=cmin, max=cmax)
    plt.figure()
    hp.orthview(im2[:,0], rot=(0,0,0), title=suffix, nest=True, cmap=cm, min=cmin, max=cmax)
    plt.figure()

Why not working?

In [ ]:
def _print_histogram(nclass, labels_train, labels_min=None, ylim=None):
    if labels_train is None:
        return
    import matplotlib.pyplot as plt
    from collections import Counter
    hist_train=Counter(labels_train)
    if labels_min is not None:
        hist_min = Counter(labels_min)
        hist_temp = hist_train - hist_min
        hist_min = hist_min - hist_train
        hist_train = hist_temp + hist_min
#         for i in range(self.nclass):
#             hist_train.append(np.sum(labels_train == i))
    labels, values = zip(*hist_train.items())
    indexes = np.asarray(labels)
#     miss = set(indexes) - set(labels)
#     if len(miss) is not 0:
#         hist_train.update({elem:0 for elem in miss})
#     labels, values = zip(*hist_train.items())
    width = 1
    plt.bar(labels, values, width)
    plt.title("labels distribution")
    plt.ylim(0,ylim)
    #plt.xticks(indexes + width * 0.5, labels)
    plt.show()

In [ ]:
_print_histogram(40, labels_test)
_print_histogram(40, predictions)
_print_histogram(40, labels_test, predictions, ylim=150)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
plt.spy(confusion_matrix(labels_test, predictions, range(40)), cmap = plt.cm.gist_heat_r)